# Homework 1

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## HBS Case

### *The Harvard Management Company and Inflation-Indexed Bonds*

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

***

# 1. HMC's Approach

**Section 1 is not graded**, and you do not need to submit your answers. But you are encouraged to think about them, and we will discuss them.

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6. Just consider how we might rewrite the optimization; don’t try to solve this extra-constrained optimization.

### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import ema_historical_return
from pypfopt.risk_models import exp_cov
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.plotting import plot_weights
from pypfopt.cla import CLA

def print(*args):
    __builtins__.print(*("%.4f" % a if isinstance(a, float) else a for a in args))
    
pd.set_option("display.precision", 5)
pd.options.display.float_format = "{:,.5f}".format

def cagr(start_p,end_p,months):
    r1 = end_p/start_p
    mon_r = (r1)**(1/months)
    cagr_ret = (mon_r-1)*12
    return cagr_ret

import warnings
warnings.filterwarnings("ignore")

***

# 2 Mean-Variance Optimization

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code. 
    
    Don't just submit code or messy numbers; submit a coherent write-up based on your work.</i>

### Data
You will need the file in the github repo, `data/multi_asset_etf_data.xlsx`.
- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill fund return, ("SHV",) which we consider as the risk-free rate.
- The data is provided in total returns, (in which case you should ignore the SHV column,) as well as excess returns, (where SHV has been subtracted from the other columns.)
- These are nominal returns-they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.
- The exhibit data that comes via Harvard with the case is unnecessary for our analysis.

### Model
We are going to analyze the problem in terms of **excess** returns.
- Thus, you will focus on the `Excess Returns` section of the lecture notes, especially the formulas on slide 50.
- Be sure to use the`excess returns` tab of the data.

### Format
In the questions below, **annualize the statistics** you report.
- Annualize the mean of monthly returns with a scaling of 12.
- Annualize the volatility of monthly returns with a scaling of $\sqrt{12}$
- The Sharpe Ratio is the mean return divided by the volatility of returns. Accordingly, we can annualize the Sharpe Ratio with a scaling of $\sqrt{12}$
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe). 

### Footnotes

#### Data File
* The case does not give time-series data, so this data has been compiled outside of the case, and it intends to represent the main asset classes under consideration via various ETFs. For details on the specific securities/indexes, check the “Info” tab of the data.

#### Risk-free rate
* In the lecture-note we considered a constant risk-free rate. It is okay that our risk-free rate changes over time, but the assumption is that investors know it’s value one-period ahead of time. Thus, at any given point in time, it is a risk-free rate for the next period. (This is often discussed as the "bank account" or "money market account" in other settings.

## 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:
$$\text{sharpe ratio of investment }i = \frac{\mux_i}{\sigma_i}$$

## 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

## 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\wtan$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\wtan$.

## 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

In [2]:
import pandas as pd
import numpy as np

df_prices = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='prices')
df_prices = df_prices.set_index('Date')

df_tot_ret = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='total returns')
df_tot_ret = df_tot_ret.set_index('Date')

df_exc_ret = pd.read_excel('multi_asset_etf_data.xlsx',sheet_name='excess returns')
df_exc_ret = df_exc_ret.set_index('Date')

display(df_prices)
display(df_tot_ret)
display(df_exc_ret)


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2009-03-31,21.54687,19.31564,18.53067,24.42950,28.47152,71.39330,15.44473,22.25579,21.24795,99.33363,60.42467,70.44867
2009-04-30,21.74066,19.29632,21.41372,27.24355,32.41367,69.43343,20.01870,27.37913,21.73415,99.38860,66.42763,69.18404
2009-05-31,22.90751,22.43512,24.82707,30.83744,33.33926,67.99111,20.47367,28.85464,22.33977,99.34173,70.31052,70.56541
2009-06-30,23.02546,21.84599,24.26859,30.40422,34.45668,67.61234,19.96463,30.06773,22.26301,99.40126,70.26450,70.70524
2009-07-31,23.74579,22.25162,26.94167,33.45723,36.84075,68.17461,22.07687,34.37486,22.60420,99.39851,75.50663,70.76740
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,22.86521,23.56000,38.82583,72.29478,73.57884,98.51483,85.27742,50.66296,29.36000,108.22806,412.93414,108.22746
2023-05-31,22.27163,22.05000,37.89314,69.39788,72.67277,97.09788,81.84364,49.69426,29.17000,108.58014,414.84033,106.93301
2023-06-30,22.45385,22.70000,39.56000,72.50000,73.96646,95.87903,86.54000,52.01878,29.83000,109.09573,441.72189,106.56959


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2009-04-30,0.00899,-0.00100,0.15558,0.11519,0.13846,-0.02745,0.29615,0.23020,0.02288,0.00055,0.09935,-0.01795
2009-05-31,0.05367,0.16266,0.15940,0.13192,0.02856,-0.02077,0.02273,0.05389,0.02787,-0.00047,0.05845,0.01997
2009-06-30,0.00515,-0.02626,-0.02249,-0.01405,0.03352,-0.00557,-0.02486,0.04204,-0.00344,0.00060,-0.00065,0.00198
2009-07-31,0.03128,0.01857,0.11015,0.10041,0.06919,0.00832,0.10580,0.14325,0.01533,-0.00003,0.07461,0.00088
2009-08-31,0.00763,-0.04036,-0.01314,0.04503,-0.01697,0.00763,0.13194,0.03341,-0.00415,0.00044,0.03694,0.00841
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,-0.00251,-0.00758,-0.00836,0.02936,0.00202,0.00815,0.00919,0.03462,0.00514,0.00310,0.01597,0.00052
2023-05-31,-0.02596,-0.06409,-0.02402,-0.04007,-0.01231,-0.01438,-0.04027,-0.01912,-0.00647,0.00325,0.00462,-0.01196
2023-06-30,0.00818,0.02948,0.04399,0.04470,0.01780,-0.01255,0.05738,0.04678,0.02263,0.00475,0.06480,-0.00340


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.00844,-0.00155,0.15503,0.11464,0.13791,-0.02801,0.29560,0.22965,0.02233,0.09879,-0.01850
2009-05-31,0.05414,0.16313,0.15987,0.13239,0.02903,-0.02030,0.02320,0.05436,0.02834,0.05892,0.02044
2009-06-30,0.00455,-0.02686,-0.02309,-0.01465,0.03292,-0.00617,-0.02546,0.04144,-0.00404,-0.00125,0.00138
2009-07-31,0.03131,0.01860,0.11017,0.10044,0.06922,0.00834,0.10583,0.14328,0.01535,0.07463,0.00091
2009-08-31,0.00719,-0.04080,-0.01357,0.04459,-0.01740,0.00720,0.13150,0.03298,-0.00459,0.03650,0.00798
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,-0.00561,-0.01068,-0.01146,0.02627,-0.00108,0.00505,0.00609,0.03152,0.00204,0.01288,-0.00258
2023-05-31,-0.02921,-0.06734,-0.02728,-0.04332,-0.01557,-0.01764,-0.04352,-0.02237,-0.00972,0.00136,-0.01521
2023-06-30,0.00343,0.02473,0.03924,0.03995,0.01305,-0.01730,0.05263,0.04203,0.01788,0.06005,-0.00815


In [3]:
X1 = list(df_exc_ret.columns)
X1.append('Annual_mean_excess_ret')
X1.append('Annual_excess_ret_vol')
X1.append('Sharpe_ratio_annual')
print(X1)
comparison_all_portfolios = pd.DataFrame(index=X1)

['BWX', 'DBC', 'EEM', 'EFA', 'HYG', 'IEF', 'IYR', 'PSP', 'QAI', 'SPY', 'TIP', 'Annual_mean_excess_ret', 'Annual_excess_ret_vol', 'Sharpe_ratio_annual']


### 1. Summary statistics

In [4]:
Exc_mean_ret = []
Vol_exc_ret = []
for i in df_exc_ret.columns:
    Exc_mean_ret.append((df_exc_ret[i].mean()*12))
    Vol_exc_ret.append(df_exc_ret[i].std()*(np.sqrt(12)))
    
mean_vol_exc = pd.DataFrame(index=df_exc_ret.columns)
mean_vol_exc['Mean_annual_ret'] = Exc_mean_ret
mean_vol_exc['Vol_annual'] = Vol_exc_ret
display(mean_vol_exc)


,Mean_annual_ret,Vol_annual
BWX,-0.00184,0.08336
DBC,0.02544,0.17898
EEM,0.06489,0.19653
EFA,0.08160,0.16599
HYG,0.06417,0.08915
IEF,0.01427,0.06241
IYR,0.12947,0.18710
PSP,0.07994,0.22739
QAI,0.01897,0.05081
SPY,0.14373,0.14768


In [5]:
mean_vol_exc['Annual_Sharpe_ratio'] = mean_vol_exc['Mean_annual_ret']/mean_vol_exc['Vol_annual']
display(mean_vol_exc.sort_values(by='Annual_Sharpe_ratio',ascending=False))

,Mean_annual_ret,Vol_annual,Annual_Sharpe_ratio
SPY,0.14373,0.14768,0.97325
HYG,0.06417,0.08915,0.71975
IYR,0.12947,0.18710,0.69200
EFA,0.08160,0.16599,0.49157
TIP,0.02232,0.05153,0.43317
QAI,0.01897,0.05081,0.37344
PSP,0.07994,0.22739,0.35155
EEM,0.06489,0.19653,0.33016
IEF,0.01427,0.06241,0.22865
DBC,0.02544,0.17898,0.14216


### 2. Descriptive analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
sns.heatmap(df_exc_ret.corr())
plt.show()

In [ ]:
df_corr = df_exc_ret.corr()
display(df_corr)
df_highest_corr = pd.DataFrame(index=df_corr.index)
corr_asset = []
value_corr = []
for i1 in df_corr.index:
    idx_2 = df_corr.loc[i1,:].nlargest(n=2)
    corr_asset.append(idx_2.index[1])
    value_corr.append(idx_2.values[1])

df_highest_corr['Security'] = corr_asset
df_highest_corr['Corr_value'] = value_corr
df_highest_corr = df_highest_corr.sort_values(by='Corr_value',ascending=False)
display(df_highest_corr)
# display(df_corr.min())

### The highest correlation is 0.896 between PSP & SPY, whereas, the lowest correlation is -0.322 between IEF and DBC 

In [ ]:
n1 = df_prices.shape[0]
sharpe_rt = dict()
CAGR_val = dict()
for i in df_prices.columns:
    st = df_prices[i][0]
    en = df_prices[i][n1-1]
    if i in ['HYG','BWX','IEF','TIP']:
        CAGR_val[i] = cagr(st,en,n1-1)
    
display(mean_vol_exc.loc[['HYG','BWX','IEF','TIP'],:])

plt.figure()
plt.bar(CAGR_val.keys(),CAGR_val.values())
plt.title('CAGR Values')
plt.show()


### The sharpe ratio of TIPS is 0.4332 which is higher than many securities in our data. The sharpe ratio of BWX is very close to zero (-0.0221), this index is designed to track fixed-rate local currency sovereign debt of investment grade countries outside US.
### However, the sharpe ratio for HYG is 0.72, whose underlying index is rules-based index consisting of US dollar-denominated high yield corporate bonds for sale in the US. 
### In comparison to IEF, TIP has much higher sharpe ratio and slightly higher CAGR. IEF has underlying index which measures the performance of public obligations of the U.S. Treasury that have a remaining maturity of greater than or equal to seven years and less than ten years.

### 3. The MV Frontier

In [ ]:
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import ema_historical_return
from pypfopt.risk_models import exp_cov
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.plotting import plot_weights
from pypfopt.cla import CLA

ef = EfficientFrontier(expected_returns=df_exc_ret.mean(), cov_matrix=df_exc_ret.cov(),weight_bounds=(-5,5))
ef1 = EfficientFrontier(expected_returns=df_exc_ret.mean(), cov_matrix=df_exc_ret.cov(),weight_bounds=(-5,5))
ef.min_volatility()
min_vol_ret = ef.portfolio_performance()[0]
min_vol_vol = ef.portfolio_performance()[1]

ef1.max_sharpe(risk_free_rate=0)
max_sharpe_ret = ef1.portfolio_performance()[0]
max_sharpe_vol = ef1.portfolio_performance()[1]

ret_ef = np.arange(0, max_sharpe_ret, 0.0001)
vol_ef = []
for i in np.arange(0, max_sharpe_ret, 0.0001):
    ef = EfficientFrontier(expected_returns=df_exc_ret.mean(), cov_matrix=df_exc_ret.cov(),weight_bounds=(-5,5))
    ef.efficient_return(i)
    vol_ef.append(ef.portfolio_performance()[1])

print(min_vol_ret)
print(min_vol_vol)
print(ef1.weights)
print(max_sharpe_ret)
print(max_sharpe_vol)


sns.set()

fig, ax = plt.subplots(figsize = [15,10])

sns.lineplot(x = vol_ef, y = ret_ef, label = "Efficient Frontier", ax = ax)
sns.scatterplot(x = [min_vol_vol], y = [min_vol_ret], ax = ax, label = "Minimum Variance Portfolio", color = "purple", s = 100)
sns.scatterplot(x = [max_sharpe_vol], y = [max_sharpe_ret], ax = ax, label = "Maximum Sharpe Portfolio", color = "green", s = 100)
# sns.lineplot(x = [0, max_sharpe_vol, 1], y = [0, max_sharpe_ret, 3.096], label = "Capital Market Line", ax = ax, color = "r")
sns.lineplot(x = [0, max_sharpe_vol], y = [0, max_sharpe_ret], label = "Capital Market Line", ax = ax, color = "r")

ax.set(xlim = [0, 0.1])
ax.set(ylim = [0, 0.05])
ax.set_xlabel("Volatility")
ax.set_ylabel("Mean Return")
plt.legend(fontsize='large')
plt.title("Efficient Frontier", fontsize = '20')

ax.figure.savefig("EffFront_big.png", dpi = 300)

In [ ]:
cov_inv = np.linalg.inv(df_exc_ret.cov())
# display(cov_inv)
num1 = cov_inv.dot(df_exc_ret.mean())
# display(num1)

ones = np.full(11,1)
den1 = (ones.T).dot(num1)
# display(den1)

cols_ret = df_exc_ret.columns
df_tot_ret_tangent = df_tot_ret[cols_ret]

weights_max_sharpe = num1/den1
# display(weights_max_sharpe)
mean_max_sharpe = (weights_max_sharpe.T).dot(df_exc_ret.mean())
vol_max_sharpe = (((weights_max_sharpe.T).dot(df_exc_ret.cov())).dot(weights_max_sharpe))**(0.5)
print('Annual Mean excess return of tangent portfolio:',mean_max_sharpe*12)
print('Annual excess return volatility of tangent portfolio:', vol_max_sharpe*(np.sqrt(12)))
print('Sharpe ratio of tangent portfolio:', (mean_max_sharpe*12)/(vol_max_sharpe*(np.sqrt(12))))

weights_dict = dict()
sharpe_dict = dict()

i=0
for i in range(len(weights_max_sharpe)):
    weights_dict[df_exc_ret.columns[i]] = weights_max_sharpe[i]
    
for j in df_exc_ret.columns:
    sharpe_dict[j] = (df_exc_ret[j].mean()*12)/(df_exc_ret[j].std()*(np.sqrt(12)))
    
print('------Weights of tangency portfolio--------------')
display(sorted(weights_dict.items(),key=lambda x: x[1]))

print('------Sharpe ratios of tangency portfolio assets--------------')
display(sorted(sharpe_dict.items(),key=lambda x: x[1]))

### The ranking of weights doesn't align with sharpe ratio rankings.

In [ ]:
tangency_portfolio1 = list(weights_max_sharpe)
tangency_portfolio1.append(mean_max_sharpe*12)
tangency_portfolio1.append(vol_max_sharpe*(np.sqrt(12)))
tangency_portfolio1.append((mean_max_sharpe*12)/(vol_max_sharpe*(np.sqrt(12))))
comparison_all_portfolios['Tang_port'] = tangency_portfolio1

In [ ]:
# -----------------------Removing TIPS-------------------

print('------------Removing TIPS----------------\n')
TICKS = ['SPY','EFA','EEM','PSP','QAI','HYG','DBC','IYR','IEF','BWX']
df_exc_ret2 = df_exc_ret[TICKS]
# display(df_exc_ret2)

cov_inv2 = np.linalg.inv(df_exc_ret2.cov())
# display(cov_inv)
num1_2 = cov_inv2.dot(df_exc_ret2.mean())
# display(num1)

ones = np.full(10,1)
den1_2 = (ones.T).dot(num1_2)
# display(den1)

weights_max_sharpe2 = num1_2/den1_2
# display(weights_max_sharpe)
mean_max_sharpe2 = (weights_max_sharpe2.T).dot(df_exc_ret2.mean())
vol_max_sharpe2 = (((weights_max_sharpe2.T).dot(df_exc_ret2.cov())).dot(weights_max_sharpe2))**(0.5)
print('Annual Mean excess return of tangent portfolio:',(mean_max_sharpe2*12))
print('Annual excess return volatility of tangent portfolio:', (vol_max_sharpe2*np.sqrt(12)))
print('Sharpe ratio of tangent portfolio:', (mean_max_sharpe2*12)/(vol_max_sharpe2*np.sqrt(12)))

# -----------------------Increasing TIPS return-------------------

print('\n------------Including TIPS with 0.0012 higher return----------------')
df_exc_ret3 = df_exc_ret
df_exc_ret3['TIP'] = df_exc_ret3['TIP']+0.0012
# display(df_exc_ret3)

cov_inv3 = np.linalg.inv(df_exc_ret3.cov())
# display(cov_inv)
num1_3 = cov_inv3.dot(df_exc_ret3.mean())
# display(num1)

ones = np.full(11,1)
den1_3 = (ones.T).dot(num1_3)
# display(den1)

weights_max_sharpe3 = num1_3/den1_3
# display(weights_max_sharpe)
mean_max_sharpe3 = (weights_max_sharpe3.T).dot(df_exc_ret3.mean())
vol_max_sharpe3 = (((weights_max_sharpe3.T).dot(df_exc_ret3.cov())).dot(weights_max_sharpe3))**(0.5)
print('Annual Mean excess return of tangent portfolio:',mean_max_sharpe3*12)
print('Annual excess return volatility of tangent portfolio:', vol_max_sharpe3*(np.sqrt(12)))
print('Sharpe ratio of tangent portfolio:', (mean_max_sharpe3*12)/(vol_max_sharpe3*(np.sqrt(12))))


In [ ]:
tangency_portfolio2 = list(weights_max_sharpe2)
tangency_portfolio2.append(0)
tangency_portfolio2.append(mean_max_sharpe2*12)
tangency_portfolio2.append(vol_max_sharpe2*(np.sqrt(12)))
tangency_portfolio2.append((mean_max_sharpe2*12)/(vol_max_sharpe2*(np.sqrt(12))))
comparison_all_portfolios['Tang_port_without_TIPS'] = tangency_portfolio2

tangency_portfolio3 = list(weights_max_sharpe3)
tangency_portfolio3.append(mean_max_sharpe3*12)
tangency_portfolio3.append(vol_max_sharpe3*(np.sqrt(12)))
tangency_portfolio3.append((mean_max_sharpe3*12)/(vol_max_sharpe3*(np.sqrt(12))))
comparison_all_portfolios['Tang_port_TIPS_higher'] = tangency_portfolio3

### Sharpe ratio of portfolio increases if TIPS is included with higher return, however, the Annual mean return reduces. 

***

# 3. Allocations

<i>This section is graded for a good-faith effort by your group. Submit your write-up- along with your supporting code.

* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\mutarg$ = 0.01.

Build the following portfolios:

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\mutarg$. Thus, the $i$ element of the weight vector is,
$$\wEW_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,
$$\wRP_i = \frac{1}{\sigma_i^2}$$

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 
$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$
but this time, use a regularized covariance matrix,
$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values. 


### Comparing

In order to compare all these allocation methods, (those above, along with the tangency portfolio obtained in the previous section,) rescale each weight vector, such that it has targeted mean return of $\mutarg$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How do these compare across the four allocation methods?

In [ ]:
# -----------------Equal weighted---------------

import random

ew_weights = np.full(11,(1/11))
print('Weights initially:',ew_weights)
ret_ew = (ew_weights.T).dot(df_exc_ret.mean())
print('Average monthly excess return:',ret_ew)
scaling_factor = 0.01/ret_ew
print('Scaling factor:',scaling_factor)
ew_weights = ew_weights*scaling_factor
print('New weights:',ew_weights)
ret_ew2 = (ew_weights.T).dot(df_exc_ret.mean())
vol_ew2 = (((ew_weights.T).dot(df_exc_ret.cov())).dot(ew_weights))**(0.5)
print('Average monthly excess return after scaling:',ret_ew2)

print('\n-----------Equi-weighted portfolio statistics----------\n')
print('Annual Mean excess return of Equi-weighted portfolio:',ret_ew2*12)
print('Annual excess return volatility of Equi-weighted portfolio:', vol_ew2*(np.sqrt(12)))
print('Sharpe ratio of Equi-weighted portfolio:', (ret_ew2*12)/(vol_ew2*(np.sqrt(12))))


In [ ]:
tangency_portfolio4 = list(ew_weights)
tangency_portfolio4.append(ret_ew2*12)
tangency_portfolio4.append(vol_ew2*(np.sqrt(12)))
tangency_portfolio4.append((ret_ew2*12)/(vol_ew2*(np.sqrt(12))))
comparison_all_portfolios['EW'] = tangency_portfolio4

In [ ]:
# ------------------Risk Parity weighted-----------------

rp_weights = 1/(df_exc_ret.std()**2)
# print(rp_weights)
# print('Weights initially:',rp_weights)
ret_rp = (rp_weights.T).dot(df_exc_ret.mean())
# print('Average monthly excess return:',ret_rp)
scaling_factor = 0.01/ret_rp
# print('Scaling factor:',scaling_factor)
rp_weights = rp_weights*scaling_factor
print('Weights:',rp_weights)
ret_rp = (rp_weights.T).dot(df_exc_ret.mean())
vol_rp = (((rp_weights.T).dot(df_exc_ret.cov())).dot(rp_weights))**(0.5)

print('\n-----------Risk parity weighted portfolio statistics----------\n')
print('Annual Mean excess return of Risk Parity weighted portfolio:',ret_rp*12)
print('Annual excess return volatility of Risk Parity weighted portfolio:', vol_rp*(np.sqrt(12)))
print('Sharpe ratio of Risk Parity weighted portfolio:', (ret_rp*12)/(vol_rp*(np.sqrt(12))))


In [ ]:
tangency_portfolio5 = list(rp_weights)
tangency_portfolio5.append(ret_rp*12)
tangency_portfolio5.append(vol_rp*(np.sqrt(12)))
tangency_portfolio5.append((ret_rp*12)/(vol_rp*(np.sqrt(12))))
comparison_all_portfolios['Risk-parity'] = tangency_portfolio5

In [ ]:
# -----------------Regularized weights---------------

cov_3 = df_exc_ret.cov()
# display(cov_3)
cov_4 = pd.DataFrame(np.diag(np.diag(cov_3)),index=cov_3.index,columns=cov_3.columns)
# display(cov_3)
# display(cov_4)
cov_reg = (cov_3.add(cov_4,fill_value=0))/2
# display(cov_reg)

cov_inv_reg = np.linalg.inv(cov_reg)
# display(cov_inv)
num1_reg = cov_inv_reg.dot(df_exc_ret.mean())
# display(num1)

ones = np.full(11,1)
den1_reg = (ones.T).dot(num1_reg)
# display(den1)

weights_max_sharpe_reg0 = num1_reg/den1_reg
display(weights_max_sharpe_reg0)
ret_reg0 = (weights_max_sharpe_reg0.T).dot(df_exc_ret.mean())
scaling_fc = 0.01/ret_reg0
print('Scaling factor:',scaling_fc)
print('---New weights---\n')
weights_max_sharpe_reg = weights_max_sharpe_reg0*scaling_fc
ret_reg = (weights_max_sharpe_reg.T).dot(df_exc_ret.mean())
display(weights_max_sharpe_reg)
vol_reg = (((weights_max_sharpe_reg.T).dot(df_exc_ret.cov())).dot(weights_max_sharpe_reg))**(0.5)

print('\n-----------Regularised weights portfolio statistics----------\n')
print('Annual Mean excess return of Regularised weighted portfolio:',ret_reg*12)
print('Annual excess return volatility of Regularised weighted portfolio:', vol_reg*(np.sqrt(12)))
print('Sharpe ratio of Regularised weighted portfolio:', (ret_reg*12)/(vol_reg*(np.sqrt(12))))


In [ ]:
tangency_portfolio6 = list(weights_max_sharpe_reg)
tangency_portfolio6.append(ret_reg*12)
tangency_portfolio6.append(vol_reg*(np.sqrt(12)))
tangency_portfolio6.append((ret_reg*12)/(vol_reg*(np.sqrt(12))))
comparison_all_portfolios['Regularized'] = tangency_portfolio6

In [ ]:
display(comparison_all_portfolios)

### The tangency portfolio which we got initially has the highest mean return and it gets even higher when TIPS was removed from the portfolio. Although, the sharpe ratio becomes higher when TIPS was included with higher return.
### Among the scaled portfolios - Regularised weights portfolio has the highest sharpe ratio which is attributed to the lowest Volatility value. 

***

# 4. Out-of-Sample Performance

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

In [7]:
df_exc_ret2 = df_exc_ret
df_exc_ret2

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.00844,-0.00155,0.15503,0.11464,0.13791,-0.02801,0.29560,0.22965,0.02233,0.09879,-0.01850
2009-05-31,0.05414,0.16313,0.15987,0.13239,0.02903,-0.02030,0.02320,0.05436,0.02834,0.05892,0.02044
2009-06-30,0.00455,-0.02686,-0.02309,-0.01465,0.03292,-0.00617,-0.02546,0.04144,-0.00404,-0.00125,0.00138
2009-07-31,0.03131,0.01860,0.11017,0.10044,0.06922,0.00834,0.10583,0.14328,0.01535,0.07463,0.00091
2009-08-31,0.00719,-0.04080,-0.01357,0.04459,-0.01740,0.00720,0.13150,0.03298,-0.00459,0.03650,0.00798
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,-0.00561,-0.01068,-0.01146,0.02627,-0.00108,0.00505,0.00609,0.03152,0.00204,0.01288,-0.00258
2023-05-31,-0.02921,-0.06734,-0.02728,-0.04332,-0.01557,-0.01764,-0.04352,-0.02237,-0.00972,0.00136,-0.01521
2023-06-30,0.00343,0.02473,0.03924,0.03995,0.01305,-0.01730,0.05263,0.04203,0.01788,0.06005,-0.00815


## 1. One-step Out-of-Sample (OOS) Performance
Let’s divide the sample to both compute a portfolio and then check its performance out of sample.
* Using only data through the end of 2021, compute the weights built in Section 3.
* Rescale the weights, (using just the in-sample data,) to set each allocation to have the same mean return of $\mutarg$.
* Using those weights, calculate the portfolio’s Sharpe ratio within that sample.
* Again using those weights, (derived using data through 2021,) calculate the portfolio’s OOS Sharpe ratio, which is based only on performance in 2022-2023.

## 2. Rolling OOS Performance

Iterate the Out-of-Sample performance every year, not just the final year. Namely,
* Start at the end of 2014, and calculate the weights through that time. Rescale them using the mean returns through that time.
* Apply the weights to the returns in the upcoming year, (2015.)
* Step forward a year in time, and recompute.
* Continue until again calculating the weights through 2022 and applying them to the returns in 2023.

Report the mean, volatility, and Sharpe from this dynamic approach for the following portfolios:
* mean-variance (tangency)
* equally-weighted
* risk-parity
* regularized

***

# 5. Without a Riskless Asset

<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

Re-do Section 2 above, but in the model without a risk-free rate.

That is, build the MV allocation using the two-part formula in the `Mean-Variance` section of the notes.
* This essentially substitutes the risk-free rate with the minimum-variance portfolio.
* Now, the allocation depends nonlinearly on the target mean return, $\mutarg$. (With a risk-free rate, we simply scale the weights up and down to achieve the mean return.)

You will find that, conceptually, the answers are very similar. 

***